# Long Run Analysis

In [ ]:
from see import DataDownload as dd


# CO-SKEL-SKY 
# 3, 60 is a good example of a locla minima from bullseye
# 12, 50, 100 exaample of better than Ground Truth.
# 15 cat's Eyes
# 36, 300 Example of not a great solution but below 0.1
# 200 Examples of good solutions but not perfect. 
# 400 Good solution
prefix = 400
folder = 'Image_Data/CO-SKEL-SKY/'

imagefiles = []
maskfiles = []
outputfiles = []

print("Adding COSKEL Data")
files = dd.getCOSKELFolderlists(folder='./Image_Data/', outputfolder='./Image_Data/Output/')
imagefiles += files[0]
maskfiles += files[1]
outputfiles += files[2]

print("Adding Sky Data")
files = dd.getSkyFolderLists(folder='./Image_Data/', outputfolder='./Image_Data/Output/')
imagefiles += files[0]
maskfiles += files[1]
outputfiles += files[2]

In [ ]:
from see import DataDownload as dd


#KOMATSUNA_output 
prefix = 20
folder = 'Image_Data/KOMATSUNA_output/'

imagefiles = []
maskfiles = []
outputfiles = []

print("Adding KOMATSUNA Data")
files = dd.getKomatsunaFolderLists(folder='./Image_Data/', outputfolder='./Image_Data/KOMATSUNA_output/')
imagefiles += files[0]
maskfiles += files[1]
outputfiles += files[2]


In [ ]:
imagefiles

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
from pathlib import Path

def evaluate_run(prefix = 0, folder='./'):
    p = Path(folder)
    filefilter = f'{prefix}_*.output'
    print(filefilter)
    filenames = list(p.glob(filefilter))

    assert(len(filenames) > 0)

    fitness = []
    pop = []
    dist = []
    for filename in filenames:
        print(filename)
        fid  = open(filename, 'r')
        lines = fid.readlines() 
        for line in lines:
            if 'BEST' in line:
                fitness.append(eval(line.split('-')[1]))
                pop.append(eval(line.split(' - ')[2]))
            if 'HIST' in line:
                dist.append(eval(line.split('-')[1]))

    fitness = np.array(fitness)
    fitness = list(fitness[:,0])
    fitness.sort(reverse=True)
    plt.plot(fitness)
    plt.xlabel('Generation')
    plt.ylabel('Fitness')
    plt.savefig(f'{filename}.png')
    return fitness, pop, dist

fitness_list, pop, dist = evaluate_run(prefix, folder=folder)

In [ ]:
for threshold in np.arange(1,0,-0.1):
    print(np.sum(np.array(fitness_list) > threshold))

In [ ]:
import imageio
from see import Segmentors
from see import JupyterGUI


print(imagefiles[prefix])
img = imageio.imread(imagefiles[prefix])
gmask = imageio.imread(maskfiles[prefix])
params = pop[-1]
print(params)
seg = Segmentors.algoFromParams(params)
mask = seg.evaluate(img)
fitness,_,_ = Segmentors.FitnessFunction(mask,gmask)
JupyterGUI.showthree(img, gmask, mask)
plt.title(f"fitness={fitness}")


In [ ]:
test = []
keys = None
for d in dist:
    test.append(list(d.values()));
    keys = list(d.keys())
    
array = np.array(test)

In [ ]:
if keys:
    plt.figure(figsize=(20,10))
    plt.subplot(211)
    plt.plot(fitness_list);
    plt.subplot(212)
    plt.plot(array);
    plt.legend(keys)

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(211)
plt.plot(fitness_list);
plt.subplot(212)
plt.plot(array);
plt.legend(keys)

In [ ]:
import copy

plt.figure(figsize=(20,10))
plt.subplot(211)
plt.plot(fitness_list);
plt.subplot(212)
ind = list(range(array.shape[0]))
shrinkray = array
index = 0
winners = []
for k in keys:
    totals = np.sum(shrinkray.T,axis=0).tolist()
    plt.bar(ind, totals, label=k);
    shrinkray = shrinkray[:,1:]

plt.legend()

winners = list(np.array(pop)[:,0])
plt.xticks(ind, winners);
    